In [2]:
from project_manager import ProjectSpec
# package import

import pandas as pd
import numpy as np
import os
import json
from pathlib import Path
from decimal import Decimal
import ansys.aedt.core

import random
import math
import shutil
import sympy as sp

In [3]:


class Parameter():

  def __init__(self):
    pass

  def _random_choice(self, X):
    # 단순 샘플링 + 반올림: value = round(min + idx*res, r)
    minv, maxv, res, r = X
    steps = int((maxv - minv) / res) + 1
    idx = np.random.randint(0, steps) if steps > 1 else 0
    return round(minv + idx * res, r)

  def setup_variable_ranges_from_json(self, json_path: Path, include_groups=('geometry','operating','cooling')):
    """
    JSON의 variables.inputs에서 [min,max,step]을 읽어 ranges 딕셔너리로 변환.
    float은 step의 소수 자릿수로 반올림 자리수를 추정, int는 0으로 설정. bool은 [0,1,1,0].
    """
    # ProjectSpec을 통해 입력 목록을 로드
    inputs = ProjectSpec.inputs(json_path, include_groups=include_groups)

    # 자리수 계산은 ProjectSpec 유틸 사용

    ranges = {}
    # 그룹 필터링: 기본은 geometry만
    if isinstance(include_groups, str):
      include_groups = (include_groups,)

    for item in inputs:
      name = item.get('name')
      typ = item.get('type')
      group = item.get('group')
      if include_groups and group not in include_groups:
        continue
      minv = item.get('min', 0)
      maxv = item.get('max', 0)
      step = item.get('step', 1 if typ == 'int' else 0.1)

      # bool 타입은 사용하지 않음 (모든 변수는 int/float)

      round_digits = 0 if typ == 'int' else ProjectSpec.decimals_from_step(step)
      ranges[name] = [minv, maxv, step, round_digits]

    return ranges


  def setup_variable_constraints_from_json(self, json_path: Path):
    # JSON의 optimization.constraint_strs 배열을 읽어 반환
    # ProjectSpec 정적 메서드 위임
    return ProjectSpec.constraint_strs(json_path)

  def select_random_variables(self, ranges=None, constraint_strs=None):
    # 랜덤 선정: 범위에 따라 값 샘플링 및 제약 반영
    if ranges is None:
      ranges = self.setup_variable_ranges_from_json("project.json")
    if constraint_strs is None:
      constraint_strs = self.setup_variable_constraints_from_json("project.json")

    # 1) 초기 샘플링
    self.w1_mm = self._random_choice(ranges['w1_mm'])
    self.l1_leg_mm = self._random_choice(ranges['l1_leg_mm'])
    self.l1_top_mm = self._random_choice(ranges['l1_top_mm'])
    self.l2_mm = self._random_choice(ranges['l2_mm'])
    self.h1_mm = self._random_choice(ranges['h1_mm'])
    self.l1_center_mm = self._random_choice(ranges['l1_center_mm'])

    self.Tx_turns = self._random_choice(ranges['Tx_turns'])
    self.Tx_width_mm = self._random_choice(ranges['Tx_width_mm'])
    self.Tx_height_mm = self._random_choice(ranges['Tx_height_mm'])
    self.Tx_space_x_mm = self._random_choice(ranges['Tx_space_x_mm'])
    self.Tx_space_y_mm = self._random_choice(ranges['Tx_space_y_mm'])
    self.Tx_preg_mm = self._random_choice(ranges['Tx_preg_mm'])

    self.Tx_layer_space_x_mm = self._random_choice(ranges['Tx_layer_space_x_mm'])
    self.Tx_layer_space_y_mm = self._random_choice(ranges['Tx_layer_space_y_mm'])

    self.Rx_turns = self._random_choice(ranges['Rx_turns'])
    self.Rx_width_mm = self._random_choice(ranges['Rx_width_mm'])
    self.Rx_height_mm = self._random_choice(ranges['Rx_height_mm'])
    self.Rx_space_x_mm = self._random_choice(ranges['Rx_space_x_mm'])
    self.Rx_space_y_mm = self._random_choice(ranges['Rx_space_y_mm'])
    self.Rx_preg_mm = self._random_choice(ranges['Rx_preg_mm'])

    self.Rx_layer_space_x_mm = self._random_choice(ranges['Rx_layer_space_x_mm'])
    self.Rx_layer_space_y_mm = self._random_choice(ranges['Rx_layer_space_y_mm'])

    self.g1_mm = self._random_choice(ranges['g1_mm'])
    self.g2_mm = self._random_choice(ranges['g2_mm'])

    # Operating variables (예: 주파수) - 사전 정의가 있으면 샘플
    if 'freq_kHz' in ranges:
      self.freq_kHz = self._random_choice(ranges['freq_kHz'])

    # Cooling variables (has_pan_bool: 0/1 정수)
    if 'has_pan_bool' in ranges:
      self.has_pan_bool = int(self._random_choice(ranges['has_pan_bool']))
    if 'pan_blow_mps' in ranges:
      self.pan_blow_mps = self._random_choice(ranges['pan_blow_mps'])
    if hasattr(self, 'has_pan_bool') and not self.has_pan_bool:
      self.pan_blow_mps = 0

    # 1.5) 누락된 키가 있으면 일반 규칙으로 보충 샘플링 (견고성)
    for name in ranges.keys():
      if not hasattr(self, name):
        setattr(self, name, self._random_choice(ranges[name]))

    # 2) SymPy 제약식 정의 (문자열) — 외부에서 주입 가능

    # 3) SymPy 파싱 준비: 변수 심볼과 로컬 함수 매핑
    var_names = list(ranges.keys())
    sym_vars = {name: sp.Symbol(name) for name in var_names}
    sym_locals = {**sym_vars, 'ceiling': sp.ceiling, 'Eq': sp.Eq, 'Implies': sp.Implies, 'Not': sp.Not}

    # 4) 제약식 파싱 및 각 제약식에 관련 변수 수집
    constraints = []
    for s in constraint_strs:
      expr = sp.sympify(s, locals=sym_locals)
      vars_in_expr = sorted({str(v) for v in getattr(expr, 'free_symbols', set())})
      constraints.append({
          'expr': expr,
          'vars': vars_in_expr,
      })

    # Cooling 제약식은 JSON에서 제거되었으므로 추가하지 않음

    # 5) 현재 값 사전 구성 함수
    def current_values_dict():
      return {name: getattr(self, name) for name in var_names}

    # 6) 제약 만족까지 반복: 실패한 제약식에 등장하는 변수만 재샘플링
    # 루프 진입 전: has_pan_bool==0이면 풍속을 0으로 고정
    if hasattr(self, 'has_pan_bool') and int(getattr(self, 'has_pan_bool')) == 0 and 'pan_blow_mps' in ranges:
      self.pan_blow_mps = 0
    max_iters = 10000
    for _ in range(max_iters):
      print(f"랜덤변수 제약조건 불만족, 재시도: {_}")
      values = current_values_dict()
      all_ok = True
      for c in constraints:
        expr_obj = c['expr']
        # expr_obj가 상수(변수 없음)면 subs 없이 바로 판정
        if not getattr(expr_obj, 'free_symbols', set()):
          ok = bool(expr_obj)
        else:
          val = expr_obj.subs(values)
          try:
            ok = bool(val)
          except TypeError:
            ok = bool(sp.simplify(val))
        if not ok:
          # 관련 변수만 재샘플링
          for vname in c['vars']:
            if vname in ranges:
              setattr(self, vname, self._random_choice(ranges[vname]))
          all_ok = False
      if all_ok:
        break

  def get_random_variable(self):
    # 범위 설정 + 랜덤 선정 순서대로 실행
    ranges = self.setup_variable_ranges_from_json("project.json")
    constraints = self.setup_variable_constraints_from_json("project.json")
    self.select_random_variables(ranges, constraints)

In [8]:
params = Parameter()
params.get_random_variable()

print(params.__dict__)

랜덤변수 제약조건 불만족, 재시도: 0
랜덤변수 제약조건 불만족, 재시도: 1
랜덤변수 제약조건 불만족, 재시도: 2
랜덤변수 제약조건 불만족, 재시도: 3
랜덤변수 제약조건 불만족, 재시도: 4
랜덤변수 제약조건 불만족, 재시도: 5
랜덤변수 제약조건 불만족, 재시도: 6
랜덤변수 제약조건 불만족, 재시도: 7
랜덤변수 제약조건 불만족, 재시도: 8
랜덤변수 제약조건 불만족, 재시도: 9
랜덤변수 제약조건 불만족, 재시도: 10
랜덤변수 제약조건 불만족, 재시도: 11
랜덤변수 제약조건 불만족, 재시도: 12
랜덤변수 제약조건 불만족, 재시도: 13
랜덤변수 제약조건 불만족, 재시도: 14
{'w1_mm': 8.1, 'l1_leg_mm': 25.2, 'l1_top_mm': 29.1, 'l2_mm': 34.9, 'h1_mm': 102.6, 'l1_center_mm': 43.7, 'Tx_turns': 2, 'Tx_width_mm': 7.2, 'Tx_height_mm': 0.7, 'Tx_space_x_mm': 8.42, 'Tx_space_y_mm': 3.24, 'Tx_preg_mm': 0.48, 'Tx_layer_space_x_mm': 5.55, 'Tx_layer_space_y_mm': 0.59, 'Rx_turns': 1, 'Rx_width_mm': 12.6, 'Rx_height_mm': 0.93, 'Rx_space_x_mm': 2.29, 'Rx_space_y_mm': 1.88, 'Rx_preg_mm': 0.62, 'Rx_layer_space_x_mm': 2.62, 'Rx_layer_space_y_mm': 3.46, 'g1_mm': 6.96, 'g2_mm': 8.51, 'freq_kHz': 55.0, 'has_pan_bool': 0, 'pan_blow_mps': 0, 'Operating_Current_in_A': 56.4}
